In [3]:
import nipype
import os,glob,sys,shutil
sys.path.append("/usr/lib/ants/")
import nipype.interfaces.fsl as fsl
import nipype.pipeline.engine as pe
import nipype.interfaces.utility as util
import nipype.interfaces.io as nio
import nipype.interfaces.ants.legacy as antsL
import nipype.interfaces.ants as ants

from IPython.display import Image

In [4]:
# DTI='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/data.nii.gz'
# DTI_B0='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain.nii.gz'
# DTI_M='/data/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain_mask.nii.gz'
# T1_M='/data/HCP_Data/HCP_BedpostData/100610/T1w/brainmask_fs.nii.gz'
# T1='/data/HCP_Data/HCP_BedpostData/100610/T1w/T1w_acpc_dc.nii.gz'
# # DTI_TO_T1='/data/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/nodif_to_T1_deformed.nii.gz
# MNI_BRAIN='/usr/share/fsl/5.0/data/standard/MNI152_T1_1mm_brain.nii.gz'
# #T1_BRAIN=''/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/T1w_acpc_dc_masked.nii.gz
# ROI_ONE='/data/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Left.nii.gz' 

# """
# Setup for DataGrabber inputs needed for the registration pipeline; thi is using the freesurfer nodif and t1 masks
# """
# # """
# datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
#         outfields=['nodif_brain_mask','xfm','invxfm','thsamples','phsamples','fsamples']),
#         name='datasource')


datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
    outfields=['nodif_brain','nodif_brain_mask','struct','struct_mask','struct_brain']),name="datasource")

# create a node to obtain the functional images
datasource.inputs.base_directory = "/data/HCP_Data/HCP_BedpostData/"
datasource.inputs.template ='*'
datasource.inputs.sort_filelist = True
datasource.inputs.field_template = dict(
    nodif_brain='%s/T1w/Diffusion/nodif_brain.nii*',
    nodif_brain_mask='%s/T1w/Diffusion/nodif_brain_mask.nii*',
    struct='%s/T1w/T1w_acpc_dc.nii*',
    struct_mask='%s/T1w/brainmask_fs.nii*', 
    struct_brain='%s/T1w/T1w_acpc_dc_masked.nii*'
)

datasource.inputs.template_args = dict(
             nodif_brain = [['subject_id']],
             nodif_brain_mask =  [['subject_id']],
             struct =  [['subject_id']],
             struct_mask = [['subject_id']],
             struct_brain = [['subject_id']]
        )

In [ ]:
!export PATH=$PATH:/usr/lib/ants
!export ANTSPATH=/usr/lib/ants

In [ ]:
subjRootDir = "/data/HCP_Data/HCP_BedpostData/"
FULL_SUBJECT_LIST = [x for x in os.listdir(subjRootDir) if os.path.isdir( subjRootDir+x+'/T1w/Diffusion.bedpostX')]
print(len(FULL_SUBJECT_LIST),"Subjects are potentially available to be processed!")

In [ ]:
datasource.inputs.raise_on_empty = False

completeSubjs = []
missingSubjs = []
for s in FULL_SUBJECT_LIST:
    datasource.inputs.subject_id = s
    results = datasource.run()

    if None not in results.outputs.get().values():
        completeSubjs.append(s)
    else:
        missingSubjs.append(s)
print("Complete:",len(completeSubjs),"MISSING Scans:",len(missingSubjs))


In [ ]:
"""
Setup for Registration  Pipeline InfoSource i.e. subjects
"""
subj_infosource = pe.Node(interface=util.IdentityInterface(fields=['subject_id']),  name="subj_infosource")
#infosource.iterables = ('subject_id', SampleSubjList)
subj_infosource.iterables = ('subject_id', completeSubjs)
### Above just converts the list of subjects into an iterable list I can connect to the next part of the pipeline

In [ ]:
## Register the B0/DTI image to the T1 --- we perhaps should have used the masked/brainonly T1??
#antsIntroduction.sh -d 3 -r $T1 i $DTI_B0 -o nodif_to_T1_ -t RI
regDTI_to_Struct = pe.Node(interface=antsL.antsIntroduction(),name='regDTI_to_Struct')
regDTI_to_Struct.inputs.dimension = 3
regDTI_to_Struct.inputs.transformation_model = 'RI'



#    pbx2 = pe.Node(interface=fsl.ProbTrackX2(), name='pbx2')

In [6]:
#antsL.
regDTI_to_Struct = antsL.antsIntroduction()
regDTI_to_Struct.inputs.dimension = 3
regDTI_to_Struct.inputs.transformation_model = 'RI'
regDTI_to_Struct.inputs.reference_image = '/data/HCP_Data/HCP_BedpostData/102008/T1w/T1w_acpc_dc.nii.gz'
regDTI_to_Struct.inputs.input_image = '/data/HCP_Data/HCP_BedpostData/102008/T1w/Diffusion/nodif_brain.nii.gz'


regDTI_to_Struct.run()


180815-20:22:00,658 interface INFO:
	 stdout 2018-08-15T20:22:00.658095:input files: checking
180815-20:22:01,961 interface INFO:
	 stdout 2018-08-15T20:22:01.961433:input files: check passed
180815-20:22:01,967 interface INFO:
	 stdout 2018-08-15T20:22:01.967598:--------------------------------------------------------------------------------------
180815-20:22:01,968 interface INFO:
	 stdout 2018-08-15T20:22:01.967598:Mapping parameters
180815-20:22:01,969 interface INFO:
	 stdout 2018-08-15T20:22:01.967598:--------------------------------------------------------------------------------------
180815-20:22:01,970 interface INFO:
	 stdout 2018-08-15T20:22:01.967598:ANTSPATH is /usr/lib/ants/
180815-20:22:01,971 interface INFO:
	 stdout 2018-08-15T20:22:01.967598:
180815-20:22:01,972 interface INFO:
	 stdout 2018-08-15T20:22:01.967598:Dimensionality:				3
180815-20:22:01,973 interface INFO:
	 stdout 2018-08-15T20:22:01.967598:Fixed image:				T1w_acpc_dc.nii.gz
180815-20:22:01,973 interfa

KeyboardInterrupt: 

In [ ]:
# antsL.antsIntroduction()
# ants.WarpImageMultiTransform()
run_hcp_reg  = pe.Workflow(name="run_hcp_reg")
run_hcp_reg.base_dir = "/data/HCP_Data/NipypeScratch/"

#samples_base_name_fxn = lambda x : x.replace('_th1samples.nii.gz','')

run_hcp_reg.connect(subj_infosource,'subject_id',datasource,'subject_id')
# ### Connect the dti_datasource to the pbx2 command
run_hcp_reg.connect( datasource,'struct_brain',regDTI_to_Struct,'reference_image')
run_hcp_reg.connect( datasource,'nodif_brain',regDTI_to_Struct,'input_image')
# runpbx2.connect( datasource,'fsamples',pbx2,'fsamples')
# runpbx2.connect( datasource,'nodif_brain_mask',pbx2,'mask')
# runpbx2.connect( datasource, ('thsamples', samples_base_name_fxn ), pbx2,'samples_base_name') ###  NOTE THIS IS A WEIRD TUPLE IS

#run_hcp_reg.run()
run_hcp_reg.run(plugin='MultiProc', plugin_args={'n_procs' : 10})

In [2]:
dagutman 46184 45332  0 16:21 ?        00:00:00 /bin/sh -c antsIntroduction.sh -d 3 -i /data/HCP_Data/HCP_BedpostData/102008/T1w/Diffusion/nodif_brain.nii.gz -o ants_ -r /data/HCP_Data/HCP_BedpostData/102008/T1w/T1w_acpc_dc.nii.gz -t RI
dagutman 46185 46184  0 16:21 ?        00:00:00 /bin/bash /usr/lib/ants//antsIntroduction.sh -d 3 -i /data/HCP_Data/HCP_BedpostData/102008/T1w/Diffusion/nodif_brain.nii.gz -o ants_ -r /data/HCP_Data/HCP_BedpostData/102008/T1w/T1w_acpc_dc.nii.gz -t RI
dagutman 46218 46185 99 16:22 ?        00:00:07 /usr/lib/ants//ANTS 3 -m PR[T1w_acpc_dc.nii.gz,nodif_brain.nii.gz,1,4] -o ants_.nii.gz -i 0 --use-Histogram-Matching --number-of-affine-iterations 10000x10000x10000x10000x10000 --MI-option 32x16000 --do-rigid true


In [ ]:
##radius is 4?

In [ ]:
### Concepts/Steps

## 3 Spaces     DTI    ---- >    T1        --->MNI
## DTI is probably "best" registered using the nodif_brain which is the first image of the "data.nii.gz" file


In [ ]:

# echo $IN $T1_M $T1

# ## NOTES PROBABLY wouLD bE QUICKER IF WE USED THE T1 IMAGE WIHT THE MASK APPLIED... TO DO!!

# export PATH=$PATH:/usr/lib/ants
# export ANTSPATH=/usr/lib/ants

# ## Get the first B0 image from the DTI data set
# #fslroi $DTI $DTI_B0 0 1 


# ## Register the B0/DTI image to the T1 --- we perhaps should have used the masked/brainonly T1??
# #antsIntroduction.sh -d 3 -r $T1 -i $DTI_B0 -o nodif_to_T1_ -t RI


# ## Need to apply the brain mask to the T1 image to speed things up
# #fslmaths $T1 -mas $T1_M $T1_MASKED


# ### Thsi registers the T1 Brain image nonlinearly to MNI Space.. and also generated the inverse warp
# #antsIntroduction.sh -d 3 -r $MNI_BRAIN  -i $T1_BRAIN -o T1_to_MNI_ -t GR

# ### note T1_To_MNI_Warp is the warp transform.. and we add two affine matrices which are also real files
# #WarpImageMultiTransform 3 $DTI_B0  nodif_to_MNI_test.nii.gz -R $MNI_BRAIN  T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt nodif_to_T1_Affine.txt


# ### THIS GOES IN REVERSE
# WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI.nii.gz -R $DTI_B0 -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz
# WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI_NN.nii.gz -R $DTI_B0 --use-NN -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz

# ## --use-NN: Use Nearest Neighbor Interpolation. 
 

END PRODUCT:   We will create a directory in Diffusion or maybe ROIs and maybe a subdirectory called DTI
    
    so /{SubjID}/T1w/ROIs/DTI_Space/Human_L_Hypothalamus.nii
       /{SubjID}/xfms/<<PUT XFMS HERE>> 
        
        

    
    

In [ ]:

# #
# #Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz
# #Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space
# #WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz

 

# #Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.

 

# #Apply transformations to get thresholded, normalized tracts into MNI space

# #WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

# #Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomi


In [ ]:
Start with the MNI-space hypothalamus / basal forebrain ROIs à want to end up with tractography in all subjects in MNI space

 

Compute rigid body transform from diffusion space to T1

antsIntroduction.sh -d 3 -r T1.nii.gz -i nodif.nii.gz -o nodif_to_T1_ -t RI

 

Compute nonlinear warp from T1 to MNI

antsIntroduction.sh -d 3 -r MNI.nii.gz -i T1.nii.gz -o T1_to_MNI_ -t GR

(I think GR (greedy Syn) should be faster than SY (Syn) which is the default and apparently I’ve been using GR with good results so far)

 

Apply transformations so you can check that it all looks OK

WarpImageMultiTransform 3 nodif.nii.gz nodif_to_MNI.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz

 

Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space

WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz

 

Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.

 

Apply transformations to get thresholded, normalized tracts into MNI space

WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomise

 

In [43]:
regDTI = ants.Registration()

regDTI.inputs.moving_image="/data/HCP_Data/HCP_BedpostData/100206/T1w/Diffusion/nodif_brain.nii.gz"
regDTI.inputs.fixed_image="/data/HCP_Data/HCP_BedpostData/100206/T1w/T1w_acpc_dc.nii.gz"
regDTI.inputs.num_threads=4
regDTI.inputs.metric = ['MI']
regDTI.inputs.transforms=['Rigid']
regDTI.inputs.smoothing_sigmas = [[4]]
regDTI.inputs.shrink_factors = [[2,1]]
regDTI.inputs.number_of_iterations=[[20]]
regDTI.inputs.convergence_threshold=[1e-08]
#regDTI.inputs.number_of_iterations=[[10,100]]
regDTI.run()
#regDTI.help()


TypeError: '_Undefined' object does not support indexing

In [111]:
# >>> import copy, pprint
from nipype.interfaces.ants import Registration
reg = Registration()
reg.inputs.fixed_image = "/data/HCP_Data/HCP_BedpostData/100206/T1w/T1w_acpc_dc_masked.nii.gz"
reg.inputs.moving_image = "/data/HCP_Data/HCP_BedpostData/100206/T1w/Diffusion/nodif_brain.nii.gz"
reg.inputs.output_transform_prefix = "100206_output_"
reg.inputs.output_warped_image = "dti_to_fa.nii.gz"
reg.inputs.transforms = ['Rigid']  ## Mandatory
reg.inputs.dimension = 3
reg.inputs.float = False
reg.inputs.interpolation = 'Linear'
reg.inputs.metric = ['MI']
reg.inputs.metric_weight = [1.0]
reg.inputs.smoothing_sigmas = [[3,2,1,0]]
reg.inputs.shrink_factors = [[8,4,2,1]]
reg.inputs.number_of_iterations= [[1000,500,250,100]]
reg.inputs.radius_or_number_of_bins=[32]
reg.inputs.radius_bins_stage_trait=[1]
reg.inputs.sampling_strategy = ['Regular']
reg.inputs.sigma_units=['vox']
reg.inputs.transform_parameters = [(0.1,)]
reg.inputs.num_threads = 16
reg.cmdline


#     coreg = pe.Node(Registration(), name='CoregAnts')
#     coreg.inputs.output_transform_prefix = 'func2highres'
#     coreg.inputs.output_warped_image = 'func2highres.nii.gz'
#     coreg.inputs.output_transform_prefix = "func2highres_"
#     coreg.inputs.transforms = ['Rigid', 'Affine']
#     coreg.inputs.number_of_iterations = [[100, 100]]*3 
#     coreg.inputs.dimension = 3
#     coreg.inputs.write_composite_transform = True
#     coreg.inputs.collapse_output_transforms = False
#     coreg.inputs.metric = ['Mattes'] * 2 
#     coreg.inputs.metric_weight = [1] * 2 
#     coreg.inputs.radius_or_number_of_bins = [32] * 2 
#     coreg.inputs.sampling_strategy = ['Regular'] * 2 
#     coreg.inputs.sampling_percentage = [0.3] * 2 
#     coreg.inputs.convergence_threshold = [1.e-8] * 2 
#     coreg.inputs.convergence_window_size = [20] * 2
#     coreg.inputs.smoothing_sigmas = [[4, 2]] * 2 
#     coreg.inputs.sigma_units = ['vox'] * 4
#     coreg.inputs.shrink_factors = [[6, 4]] + [[3, 2]]
#     coreg.inputs.use_estimate_learning_rate_once = [True] * 2
#     coreg.inputs.use_histogram_matching = [False] * 2 
#     coreg.inputs.initial_moving_transform_com = True

# dgutman  35042 35041 99 11:25 pts/8    00:00:26 antsRegistration --dimensionality 3 --float 0 --output [/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/sampleOutput/NN_DTI_to_Struct_11111_,/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/sampleOutput/NN_DTI_to_Struct_11111_Warped.nii.gz] --interpolation NearestNeighbor --winsorize-image-intensities [0.005,0.995] --use-histogram-matching 0 --transform Rigid[0.1] --metric MI[/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/T1w_acpc_dc_masked.nii.gz,/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif.nii.gz,1,32,Regular,0.25] --convergence [1000x500x250x100,1e-6,10] --shrink-factors 8x4x2x1 --smoothing-sigmas 3x2x1x0vox
# dgutman  44164 26213  0 11:25 pts/8    00:00:00 grep --color=auto ants
reg.run()

In [ ]:

thisfolder=${PWD}/sampleOutput
sub=11111 

antsRegistration --dimensionality 3 --float 0 \
--output [$thisfolder/NN_DTI_to_Struct_${sub}_,$thisfolder/NN_DTI_to_Struct_${sub}_Warped.nii.gz] \
--interpolation NearestNeighbor \
--winsorize-image-intensities [0.005,0.995] \
--use-histogram-matching 0 \
--transform Rigid[0.1] \
--metric MI[$T1_BRAIN,$DTI_B0,1,32,Regular,0.25] \
--convergence [1000x500x250x100,1e-6,10] \
--shrink-factors 8x4x2x1 \
--smoothing-sigmas 3x2x1x0vox 

#reg.inputs.convergence_threshold = [1.e-6]
#reg.inputs.convergence_window_size = [10]
# reg.inputs.use_histogram_matching = False
# reg.inputs.winsorize_upper_quantile = 0.995
# reg.inputs.winsorize_lower_quantile = 0.005

# >>> reg.inputs.metric_weight = [1]*2 # Default (value ignored currently by ANTs)
# >>> reg.inputs.radius_or_number_of_bins = [32]*2
# >>> reg.inputs.sampling_strategy = ['Random', None]
# >>> reg.inputs.sampling_percentage = [0.05, None]
# >>> reg.inputs.convergence_window_size = [20]*2
# >>> reg.inputs.sigma_units = ['vox'] * 2
# >>> reg.inputs.use_estimate_learning_rate_once = [True, True]
# >>> reg.inputs.use_histogram_matching = [True, True] # This is the default
# >>> reg.inputs.output_warped_image = 'output_warped_image.nii.gz'
# >>> reg.inputs.write_composite_transform = True
# >>> reg.inputs.collapse_output_transforms = False
# >>> reg.inputs.initialize_transforms_per_stage = False

# >>> reg.cmdline
# 'antsRegistration --collapse-output-transforms 0 --dimensionality 3 --initial-moving-transform [ trans.mat, 0 ] --initialize-transforms-per-stage 0 --interpolation Linear --output [ output_, output_warped_image.nii.gz ] --transform Affine[ 2.0 ] --metric Mattes[ fixed1.nii, moving1.nii, 1, 32, Random, 0.05 ] --convergence [ 1500x200, 1e-



In [ ]:

metric: (a list of items which are 'CC' or 'MeanSquares' or 'Demons'
         or 'GC' or 'MI' or 'Mattes' or a list of items which are 'CC' or
         'MeanSquares' or 'Demons' or 'GC' or 'MI' or 'Mattes')
        the metric(s) to use for each stage. Note that multiple metrics per
        stage are not supported in ANTS 1.9.1 and earlier.
metric_weight: (a list of items which are a float or a list of items
         which are a float, nipype default value: [1.0])
        the metric weight(s) for each stage. The weights must sum to 1 per
        stage.
        requires: metric
moving_image: (a list of items which are an existing file name)
        Image that will be registered to the space of fixed_image. This is
        theimage on which the transformations will be applied to
shrink_factors: (a list of items which are a list of items which are
         an integer (int or long))
smoothing_sigmas: (a list of items which are a list of items which
         are a float)
transforms: (a list of items which are 'Rigid' or 'Affine' or
         'CompositeAffine' or 'Similarity' or 'Translation' or 'BSpline' or
         'GaussianDisplacementField' or 'TimeVaryingVelocityField' or
         'TimeVaryingBSplineVelocityField' or 'SyN' or 'BSplineSyN' or
         'Exponential' or 'BSplineExponential')
        flag: %s

In [ ]:

# >>> import copy, pprint
# >>> from nipype.interfaces.ants import Registration
# >>> reg = Registration()
# >>> reg.inputs.fixed_image = 'fixed1.nii'
# >>> reg.inputs.moving_image = 'moving1.nii'
# >>> reg.inputs.output_transform_prefix = "output_"
# >>> reg.inputs.initial_moving_transform = 'trans.mat'
# >>> reg.inputs.transforms = ['Affine', 'SyN']
# >>> reg.inputs.transform_parameters = [(2.0,), (0.25, 3.0, 0.0)]
# >>> reg.inputs.number_of_iterations = [[1500, 200], [100, 50, 30]]
# >>> reg.inputs.dimension = 3
# >>> reg.inputs.write_composite_transform = True
# >>> reg.inputs.collapse_output_transforms = False
# >>> reg.inputs.initialize_transforms_per_stage = False
# >>> reg.inputs.metric = ['Mattes']*2
# >>> reg.inputs.metric_weight = [1]*2 # Default (value ignored currently by ANTs)
# >>> reg.inputs.radius_or_number_of_bins = [32]*2
# >>> reg.inputs.sampling_strategy = ['Random', None]
# >>> reg.inputs.sampling_percentage = [0.05, None]
# >>> reg.inputs.convergence_threshold = [1.e-8, 1.e-9]
# >>> reg.inputs.convergence_window_size = [20]*2
# >>> reg.inputs.smoothing_sigmas = [[1,0], [2,1,0]]
# >>> reg.inputs.sigma_units = ['vox'] * 2
# >>> reg.inputs.shrink_factors = [[2,1], [3,2,1]]
# >>> reg.inputs.use_estimate_learning_rate_once = [True, True]
# >>> reg.inputs.use_histogram_matching = [True, True] # This is the default
# >>> reg.inputs.output_warped_image = 'output_warped_image.nii.gz'
# >>> reg.cmdline
# 'antsRegistration --collapse-output-transforms 0 --dimensionality 3 --initial-moving-transform [ trans.mat, 0 ] --initialize-transforms-per-stage 0 --interpolation Linear --output [ output_, output_warped_image.nii.gz ] --transform Affine[ 2.0 ] --metric Mattes[ fixed1.nii, moving1.nii, 1, 32, Random, 0.05 ] --convergence [ 1500x200, 1e-08, 20 ] --smoothing-sigmas 1.0x0.0vox --shrink-factors 2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --transform SyN[ 0.25, 3.0, 0.0 ] --metric Mattes[ fixed1.nii, moving1.nii, 1, 32 ] --convergence [ 100x50x30, 1e-09, 20 ] --smoothing-sigmas 2.0x1.0x0.0vox --shrink-factors 3x2x1 --use-estimate-learning-rate-once 1 --use-histogram-matching 1 --winsorize-image-intensities [ 0.0, 1.0 ]  --write-composite-transform 1'
# >>> reg.run()  
# seedMask = "/data/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Bilat.nii.gz"
# seedMaskList = glob.glob('/data/HCP_Data/EHECHT_ROIS/Hu*')
# debug_output = False

# if debug_output:
#     pbx2 = fsl.ProbTrackX2()
#     pbx2.inputs.thsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_th1samples.nii.gz'
#     pbx2.inputs.phsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_ph1samples.nii.gz'
#     pbx2.inputs.fsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_f1samples.nii.gz'
#     pbx2.inputs.mask = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/nodif_brain_mask.nii.gz'
#     pbx2.inputs.samples_base_name = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged'
# else:
#     pbx2 = pe.Node(interface=fsl.ProbTrackX2(), name='pbx2')

    
# pbx2.inputs.c_thresh = 0.2   # -c 0.2   F cutoff
# pbx2.inputs.n_steps = 2000   # -S 2000
# pbx2.inputs.step_length = 0.5 # --steplength=0.5
# pbx2.inputs.n_samples = 5000  # -P 5000
# pbx2.inputs.opd = True
# pbx2.inputs.loop_check = True
# # pbx2.inputs.fibst = True

# pbx2.iterables = ("seed", seedMaskList)

# # pbx2.inputs.onewaycondition = True # --onewaycondition
# pbx2.inputs.correct_path_distribution = True # -pd  i.e. distance correction
# #pbx2.inputs.lrtarget3= targetMask  ## This is a hack for now
# pbx2.inputs.seed = seedMask
# #pbx2.inputs.omatrix3 = True   # --omatrix3
# #pbx2.inputs.target3 = targetMask # --target3=$WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_GM.nii.gz "; 

# ### #FIBTHRESH APPEARS MISSING FROM INTERFACE???
# if debug_output:
#     print( pbx2.cmdline)
#     print("WE WILL BE PROCESSING THE FOLLOWING ROIS")
#     print([os.path.basename(x) for x in seedMaskList])
#     pbx2.inputs.os2t = True

In [ ]:
# # """
# # Setup for DataGrabber inputs needed for probtrackx2
# # """
# datasource = pe.Node(interface=nio.DataGrabber(infields=['subject_id'],
#         outfields=['nodif_brain_mask','xfm','invxfm','thsamples','phsamples','fsamples']),
#         name='datasource')
# # create a node to obtain the functional images
# datasource.inputs.base_directory = "/data/HCP_Data/HCP_BedpostData/"
# datasource.inputs.template ='*'
# datasource.inputs.sort_filelist = True
# datasource.inputs.field_template = dict(
#     thsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
#     fsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
#     phsamples='%s/T1w/Diffusion.bedpostX/merged_%s.nii*',
#     nodif_brain_mask='%s/T1w/Diffusion.bedpostX/%s.nii*',
#     nodif_brain='%s/T1w/Diffusion/%s.nii*',
#     nodif='%s/T1w/Diffusion/%s.nii*',

#     xfm='%s/MNINonLinear/xfms/%s.nii*',
#     invxfm='%s/MNINonLinear/xfms/%s.nii*'
#     )

# datasource.inputs.template_args = dict(
#              thsamples = [['subject_id','th1samples']],
#              phsamples =  [['subject_id','ph1samples']],
#              fsamples =  [['subject_id','f1samples']],
#              nodif_brain_mask = [['subject_id','nodif_brain_mask']],
#              nodif_brain = [['subject_id','nodif_brain']],
#              nodif = [['subject_id','nodif']],
#              xfm = [['subject_id','acpc_dc2standard']],
#              invxfm = [['subject_id', 'standard2acpc_dc']]           
#         )

In [ ]:
seedMask = "/data/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Bilat.nii.gz"
seedMaskList = glob.glob('/data/HCP_Data/EHECHT_ROIS/Hu*')
debug_output = False

if debug_output:
    pbx2 = fsl.ProbTrackX2()
    pbx2.inputs.thsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_th1samples.nii.gz'
    pbx2.inputs.phsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_ph1samples.nii.gz'
    pbx2.inputs.fsamples=subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged_f1samples.nii.gz'
    pbx2.inputs.mask = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/nodif_brain_mask.nii.gz'
    pbx2.inputs.samples_base_name = subjRootDir+SampleSubjList[0]+'/T1w/Diffusion.bedpostX/merged'
else:
    pbx2 = pe.Node(interface=fsl.ProbTrackX2(), name='pbx2')

    
pbx2.inputs.c_thresh = 0.2   # -c 0.2   F cutoff
pbx2.inputs.n_steps = 2000   # -S 2000
pbx2.inputs.step_length = 0.5 # --steplength=0.5
pbx2.inputs.n_samples = 5000  # -P 5000
pbx2.inputs.opd = True
pbx2.inputs.loop_check = True
# pbx2.inputs.fibst = True

pbx2.iterables = ("seed", seedMaskList)

# pbx2.inputs.onewaycondition = True # --onewaycondition
pbx2.inputs.correct_path_distribution = True # -pd  i.e. distance correction
#pbx2.inputs.lrtarget3= targetMask  ## This is a hack for now
pbx2.inputs.seed = seedMask
#pbx2.inputs.omatrix3 = True   # --omatrix3
#pbx2.inputs.target3 = targetMask # --target3=$WORKINGDATAPATH/All_Foxes_FA_to_Unselected-Template_GM.nii.gz "; 

### #FIBTHRESH APPEARS MISSING FROM INTERFACE???
if debug_output:
    print( pbx2.cmdline)
    print("WE WILL BE PROCESSING THE FOLLOWING ROIS")
    print([os.path.basename(x) for x in seedMaskList])
    pbx2.inputs.os2t = True

In [ ]:
probtrackx2  --fibthresh=0.1 --randfib=0 

In [ ]:
#runpbx.connect(subj_infosource,'subject_id',datasource,'subject_id')
runpbx2  = pe.Workflow(name="runpbx2")
runpbx2.base_dir = "/data/HCP_Data/NipypeScratch/"

samples_base_name_fxn = lambda x : x.replace('_th1samples.nii.gz','')


runpbx2.connect(subj_infosource,'subject_id',datasource,'subject_id')
# ### Connect the dti_datasource to the pbx2 command
runpbx2.connect( datasource,'thsamples',pbx2,'thsamples')
runpbx2.connect( datasource,'phsamples',pbx2,'phsamples')
runpbx2.connect( datasource,'fsamples',pbx2,'fsamples')
runpbx2.connect( datasource,'nodif_brain_mask',pbx2,'mask')
runpbx2.connect( datasource, ('thsamples', samples_base_name_fxn ), pbx2,'samples_base_name') ###  NOTE THIS IS A WEIRD TUPLE IS

#runpbx2.run()
runpbx2.run(plugin='MultiProc', plugin_args={'n_procs' : 30})

In [ ]:
DTI=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/data.nii.gz
DTI_B0=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain.nii.gz
DTI_M=/HCP_Data/HCP_BedpostData/100610/T1w/Diffusion/nodif_brain_mask.nii.gz
T1_M=/HCP_Data/HCP_BedpostData/100610/T1w/brainmask_fs.nii.gz
T1=/HCP_Data/HCP_BedpostData/100610/T1w/T1w_acpc_dc.nii.gz
DTI_TO_T1=/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/nodif_to_T1_deformed.nii.gz
MNI_BRAIN=/usr/share/fsl/5.0/data/standard/MNI152_T1_1mm_brain.nii.gz 

T1_BRAIN=/HCP_Data/Scripts/GutmanLabNipypeWorkflows/HCP_Pipelines/sampleAntsRegData/T1w_acpc_dc_masked.nii.gz

ROI_ONE=/HCP_Data/EHECHT_ROIS/Human_Hypothalamus_Left.nii.gz  


echo $IN $T1_M $T1

## NOTES PROBABLY wouLD bE QUICKER IF WE USED THE T1 IMAGE WIHT THE MASK APPLIED... TO DO!!

export PATH=$PATH:/usr/lib/ants
export ANTSPATH=/usr/lib/ants

## Get the first B0 image from the DTI data set
#fslroi $DTI $DTI_B0 0 1 


## Register the B0/DTI image to the T1 --- we perhaps should have used the masked/brainonly T1??
#antsIntroduction.sh -d 3 -r $T1 i $DTI_B0 -o nodif_to_T1_ -t RI


## Need to apply the brain mask to the T1 image to speed things up
#fslmaths $T1 -mas $T1_M $T1_MASKED


### Thsi registers the T1 Brain image nonlinearly to MNI Space.. and also generated the inverse warp
#antsIntroduction.sh -d 3 -r $MNI_BRAIN  -i $T1_BRAIN -o T1_to_MNI_ -t GR

### note T1_To_MNI_Warp is the warp transform.. and we add two affine matrices which are also real files
#WarpImageMultiTransform 3 $DTI_B0  nodif_to_MNI_test.nii.gz -R $MNI_BRAIN  T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt nodif_to_T1_Affine.txt


### THIS GOES IN REVERSE
WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI.nii.gz -R $DTI_B0 -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz
WarpImageMultiTransform 3 $ROI_ONE ROI_ONE_to_subjDTI_NN.nii.gz -R $DTI_B0 --use-NN -i nodif_to_T1_Affine.txt -i T1_to_MNI_Affine.txt  T1_to_MNI_InverseWarp.nii.gz

## --use-NN: Use Nearest Neighbor Interpolation. 
 

#
#Then check that nodif-to-MNI mappings look good: slicesdir –p MNI_template.nii.gz Subj_*_nodif_to_MNI.nii.gz
#Apply INVERSE transformation from MNI-space ROIs to individual subjects’ diffusion space
#WarpImageMultiTransform 3 template_ROI.nii.gz SubjX_ROI.nii.gz –R nodif.nii.gz –i DTI_to_T1_Affine.txt –i T1_to_MNI_Affine.txt T1_to_MNI_InverseWarp.nii.gz

 

#Run probtrackx using native-space ROIs… this will produce a native-space fdt_paths.nii.gz file.  Threshold and normalize in native space.

 

#Apply transformations to get thresholded, normalized tracts into MNI space

#WarpImageMultiTransform 3 fdt_paths_thresh_norm.nii.gz SubjX_fdt_paths_thresh_norm_in_MNIspace.nii.gz -R MNI.nii.gz T1_to_MNI_Warp.nii.gz T1_to_MNI_Affine.txt DTI_to_T1_Affine.txt

 

#Fslmerge –t Subj*_fdt_paths_thresh_norm_in_MNIspace.nii.gz à this becomes the input for randomi

